# [실습5] Vector DB 캐싱

## 실습 목표
---
[실습5] Vector DB 인덱싱 시간을 절약하기 위한 캐싱 기법을 사용해 봅니다.

## 실습 목차
---

1. **Vector DB 임베딩 캐싱:** Vector DB를 캐싱하고 저장 및 불러오는 기능을 구현합니다.

## 실습 개요
---
챗봇의 기능을 고도화 하기 위한 전략 중 하나인 Vector DB를 캐싱하는 방법을 학습합니다.

## 0. 환경 설정
- 필요한 라이브러리를 불러옵니다.
- 이번에는 gemini와 gpt 모두 활용하여 실습합니다.

In [14]:
import os
import time

from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS

from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI


from langchain_community.chat_models import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

In [15]:
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = ""

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = ""


gemini 모델을 사용하는 ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings 객체를 생성합니다.

In [16]:
gemini_llm = ChatGoogleGenerativeAI(model="gemini-pro")
gemini_route_llm = ChatGoogleGenerativeAI(model="gemini-pro")
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

gpt-4o-mini 모델을 사용하는 ChatOpenAI, OpenAIEmbeddings 객체를 생성합니다.

In [ ]:
gpt_llm = ChatOpenAI(model="gpt-4o-mini")
gpt_route_llm = ChatOpenAI(model="gpt-4o-mini", format="json")
gpt_embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

data_dir = "../data"

## 1. Vector DB 임베딩 캐싱

시장 조사 문건을 불러와서 `GoogleGenerativeAIEmbeddings, OpenAIEmbeddings`를 활용해 벡터로 변환하고, 각각 FAISS DB를 활용하여 저장했습니다.
따로 하는 이유는 embedding 모델이 다르기 때문에 벡터 값이 달라지기 때문입니다.

- 출처: 국토교통부의 교통 분야 3대 혁신 전략 보도자료
  - https://www.molit.go.kr/USR/NEWS/m_71/dtl.jsp?lcmspage=1&id=95089349

In [ ]:
%%time

# 시장 조사 문건을 불러옵니다.
doc_path = os.path.join(data_dir, '교통_3대_혁신_전략.pdf')
loader = PyPDFLoader(doc_path)
docs = loader.load()

# embedding은 gpt를 통해 진행
gpt_vectorstore = FAISS.from_documents(
    docs,
    embedding=gpt_embeddings
)

db_retriever = gpt_vectorstore.as_retriever()

### [TODO] FAISS로 document를 불러오고 gemini_vectorstore 변수를 만들어보세요.

In [ ]:
%%time

# 시장 조사 문건을 불러옵니다.
doc_path = os.path.join(data_dir, '교통_3대_혁신_전략.pdf')
loader = PyPDFLoader(doc_path)
docs = loader.load()

# embedding은 gemini를 통해 진행
gemini_vectorstore = _______________________________

db_retriever = gemini_vectorstore.as_retriever()

문서 하나를 불러오는데 약 2~3분 정도 소요되었습니다.<br>
만약 사용하고자 하는 문서가 매우 많다면 챗봇을 사용하려 할 때 마다 문서를 불러오면서 많은 시간이 낭비될 것입니다.

이를 방지하기 위해, 임베딩을 마친 Vector DB를 캐싱하는 방법과, 별도로 저장하는 방법을 학습해 봅시다.

### 1.1 임베딩 캐싱

VectorStore의 `save_local` 메서드를 활용해서 임베딩 완료된 DB를 별도의 파일로 추출할 수 있으며, `load_local` 메서드를 활용해서 다시 불러올 수 있습니다.

In [20]:
gemini_vectorstore.save_local("./.cache/gemini_vectorstore/traffic")
gpt_vectorstore.save_local("./.cache/gpt_vectorstore/traffic")


In [ ]:
%%time

gpt_new_vectorstore = FAISS.load_local(
    "./.cache/gpt_vectorstore/traffic",
    embeddings=gpt_embeddings,
    allow_dangerous_deserialization=True,
)

db_retriever = gpt_new_vectorstore.as_retriever()

### [TODO] FAISS로 local에 저장한 벡터값을 불러오고 gemini_vectorstore 변수를 만들어보세요.

In [ ]:
%%time

gemini_new_vectorstore = _______________________________

db_retriever = gemini_new_vectorstore.as_retriever()

불러오는 시간이 크게 단축된 것을 확인할 수 있습니다.

`load_local` 메서드를 확인하면 `allow_dangerous_deserialization` 인자가 True로 설정되어 있습니다.

FAISS DB는 로컬 파일로 저장할 때 pickle을 사용합니다. pickle 라이브러리의 보안 취약성으로 인해, Product에는 임의의 사용자가 제공한 pkl 파일을 사용하지 않는 것을 강력히 권장합니다.<br> 즉, 개발자가 서버 단에 적용한 것이 확실한 파일만 불러오거나, pickle을 사용하지 않는 ChromaDB를 사용하는 등 보안 정책을 적용해야 합니다.

## 1.2 응답 캐싱

응답을 캐싱을 사용하여, 언어 모델의 응답을 저장하여 재사용할 수도 있습니다. 이를 통해 반복적인 질문에 대해 비용을 절약하고 응답 시간을 대폭 줄일 수 있습니다. 이렇게 하면 다음과 같은 이점이 있습니다:

- __비용 절약__: 동일한 질문에 대해 LLM을 반복 호출하지 않으므로 API 호출 비용을 절감할 수 있습니다.
- __빠른 응답__: 캐시에 저장된 결과를 즉시 반환할 수 있어, 응답 시간이 매우 빨라집니다.

In [ ]:
import time
from langchain.globals import set_llm_cache
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())

start_time = time.time()

response = gpt_llm.invoke("교통비 부담을 줄이기 위한 K-패스에 대해 설명해줘.")

time_passed = time.time() - start_time

print(f"답변: {response.content}, \n소요 시간: {round(time_passed, 2)} 초")

In [ ]:
import time

start_time = time.time()

response = gpt_llm.invoke("교통비 부담을 줄이기 위한 K-패스에 대해 설명해줘.")

time_passed = time.time() - start_time

print(f"답변: {response.content}, \n소요 시간: {round(time_passed, 2)} 초")

### [TODO] gemini로 gpt로 했던 응답 캐싱 과정을 반복해보세요.

LLM을 처음 호출할 때 대비 캐시를 사용해서 동일한 질문을 다시 물어보면 소요 시간이 크게 줄어드는 것을 확인할 수 있습니다.